## The Delegation Run

If classes are objects what is the difference between types and instances?

When I talk about "my cat" I am referring to a concrete instance of the "cat" concept, which is a _subtype_ of "animal". So, despite being both objects, while types can be _specialized_, instances cannot.

Usually an object B is said to be a specialization of an object A when:

* B has all the features of A
* B can provide new features
* B can perform some or all the tasks performed by A in a different way

Those targets are very general and valid for any system and the key to achieve them with the maximum reuse of already existing components is _delegation_. Delegation means that an object shall perform only what it knows best, and leave the rest to other objects.

Delegation can be implemented with two different mechanisms: _composition_ and _inheritance_. Sadly, very often only inheritance is listed among the pillars of OOP techniques, forgetting that it is an implementation of the more generic and fundamental mechanism of delegation; perhaps a better nomenclature for the two techniques could be _explicit delegation_ (composition) and _implicit delegation_ (inheritance).

Please note that, again, when talking about composition and inheritance we are talking about focusing on a behavioural or structural delegation. Another way to think about the difference between composition and inheritance is to consider if the object _knows_ who can satisfy your request or if the object _is_ the one that satisfy the request.

**Please, please, please do not forget composition**: in many cases, composition can lead to simpler systems, with benefits on maintainability and changeability. 

Usually composition is said to be a very generic technique that needs no special syntax, while inheritance and its rules are strongly dependent on the language of choice. Actually, the strong dynamic nature of Python softens the boundary line between the two techniques.

## Inheritance Now

In Python a class can be declared as an _extension_ of one or more different classes, through the _class inheritance_ mechanism. The child class (the one that inherits) has the same internal structure of the parent class (the one that is inherited), and for the case of multiple inheritance the language has very specific rules to manage possible conflicts or redefinitions among the parent classes. A very simple example of inheritance is

In [37]:
class Door:
    colour = 'brown'

    def __init__(self, number, status):
        self.number = number
        self.status = status

    @classmethod
    def knock(self):
        print("Knock!")

    @classmethod
    def paint(self, colour):
        self.colour = colour
        
    @classmethod
    def paint2(self, number):
        self.number = number

    def open(self):
        self.status = 'open'
        
    def close(self):
        self.status = 'closed'
        
class SecurityDoor(Door,):
    colour = 'pink'

In [2]:
SecurityDoor.__mro__

(__main__.SecurityDoor, __main__.Door, object)

In [3]:
t = Door(1, 'o')

In [4]:
t.knock()

Knock!


In [5]:
t.paint('red')

In [6]:
vars(t)

{'number': 1, 'status': 'o'}

In [7]:
t.colour

'red'

In [8]:
Door.paint('yello')

In [9]:
t.paint2(3)

In [10]:
vars(t)

{'number': 1, 'status': 'o'}

In [11]:
Door.number

3

where we declare a new class `SecurityDoor` that, at the moment, is a perfect copy of the `Door` class. Let us investigate what happens when we access attributes and methods. First we instance the class

In [12]:
sdoor = SecurityDoor(1, 'closed')

The first check we can do is that class attributes are still global and shared

In [13]:
print(SecurityDoor.colour is Door.colour)

False


In [14]:
print(sdoor.colour is Door.colour)

False


This shows us that Python tries to resolve instance members not only looking into the class the instance comes from, but also investigating the parent classes. In this case `sdoor.colour` becomes `SecurityDoor.colour`, that in turn becomes `Door.colour`. `SecurityDoor` _is_ a `Door`.

If we investigate the content of `__dict__` we can catch a glimpse of the inheritance mechanism in action

In [15]:
print(sdoor.__dict__)

{'number': 1, 'status': 'closed'}


In [16]:
print(type(sdoor.__class__.__dict__))
print(sdoor.__class__.__dict__)

<class 'mappingproxy'>
{'__module__': '__main__', 'colour': 'pink', '__doc__': None}


In [17]:
vars(Door)

mappingproxy({'__module__': '__main__',
              'colour': 'yello',
              '__init__': <function __main__.Door.__init__(self, number, status)>,
              'knock': <classmethod at 0x1d4e9836e80>,
              'paint': <classmethod at 0x1d4e9836eb8>,
              'paint2': <classmethod at 0x1d4e9836ef0>,
              'open': <function __main__.Door.open(self)>,
              'close': <function __main__.Door.close(self)>,
              '__dict__': <attribute '__dict__' of 'Door' objects>,
              '__weakref__': <attribute '__weakref__' of 'Door' objects>,
              '__doc__': None,
              'number': 3})

In [18]:
print(type(Door.__dict__))
print(Door.__dict__)

<class 'mappingproxy'>
{'__module__': '__main__', 'colour': 'yello', '__init__': <function Door.__init__ at 0x000001D4E9819400>, 'knock': <classmethod object at 0x000001D4E9836E80>, 'paint': <classmethod object at 0x000001D4E9836EB8>, 'paint2': <classmethod object at 0x000001D4E9836EF0>, 'open': <function Door.open at 0x000001D4E9819F28>, 'close': <function Door.close at 0x000001D4E98196A8>, '__dict__': <attribute '__dict__' of 'Door' objects>, '__weakref__': <attribute '__weakref__' of 'Door' objects>, '__doc__': None, 'number': 3}


As you can see the content of `__dict__` for `SecurityDoor` is very narrow compared to that of `Door`. The inheritance mechanism takes care of the missing elements by climbing up the classes tree. Where does Python get the parent classes? A class always contains a `__bases__` tuple that lists them

In [19]:
print(SecurityDoor.__bases__)

(<class '__main__.Door'>,)


So an example of what Python does to resolve a class method call through the inheritance tree is

In [20]:
print(sdoor.__class__.__bases__[0].__dict__['knock'].__get__(sdoor))

<bound method Door.knock of <class '__main__.SecurityDoor'>>


In [21]:
t = sdoor.__class__.__bases__[0].__dict__['knock']

In [22]:
s = t.__get__(sdoor)

In [24]:
def x(self):
    return 1

In [25]:
Door.x = x

In [37]:
print(sdoor.knock)

<bound method type.knock of <class '__main__.SecurityDoor'>>


In [1]:
import matplotlib

In [2]:
len(dir(matplotlib))

109

In [3]:
import matplotlib.pyplot as plt
#matplotlib 에 동적으로 추가함
## __slot__으로 막을 수 있다.

In [4]:
len(dir(matplotlib))

172

In [5]:
class B:
    def __repr__(self): #관례적으로 raw 포맷을 쓴다
        return '문근영'
    def __str__(self):
        return '문근영2'

In [6]:
b = B()

In [7]:
b

문근영

In [8]:
print(b)

문근영2


Please note that this is just an example that does not consider multiple inheritance.

Let us try now to override some methods and attributes. In Python you can _override_ (redefine) a parent class member simply by redefining it in the child class.

In [42]:
class SecurityDoor(Door):
    colour = 'gray'
    locked = True
    
    def open(self): ## override : 이 클래스 안에서만 쓸 수 있다.
        if not self.locked:
            self.status = 'open'

In [43]:
SecurityDoor.__mro__

(__main__.SecurityDoor, __main__.Door, object)

As you can forecast, the overridden members now are present in the `__dict__` of the `SecurityDoor` class

In [39]:
print(type(SecurityDoor.__dict__))
print(SecurityDoor.__dict__)

<class 'mappingproxy'>
{'__doc__': None, 'locked': True, '__module__': '__main__', 'open': <function SecurityDoor.open at 0xb48ee104>, 'colour': 'gray'}


So when you override a member, the one you put in the child class is used instead of the one in the parent class simply because the former is found before the latter while climbing the class hierarchy. This also shows you that Python does not implicitly call the parent implementation when you override a method. So, overriding is a way to block implicit delegation.

If we want to call the parent implementation we have to do it explicitly. In the former example we could write

In [40]:
class SecurityDoor(Door):
    colour = 'gray'
    locked = True
    
    def open(self):
        if self.locked:
            return
        Door.open(self)

You can easily test that this implementation is working correctly.

In [41]:
sdoor = SecurityDoor(1, 'closed')
print(sdoor.status)

closed


In [42]:
sdoor.open()
print(sdoor.status)

closed


In [43]:
sdoor.locked = False
sdoor.open()
print(sdoor.status)

open


This form of explicit parent delegation is heavily discouraged, however.

The first reason is because of the very high coupling that results from explicitly naming the parent class again when calling the method. _Coupling_, in the computer science lingo, means to link two parts of a system, so that changes in one of them directly affect the other one, and is usually avoided as much as possible. In this case if you decide to use a new parent class you have to manually propagate the change to every method that calls it. Moreover, since in Python the class hierarchy can be dynamically changed (i.e. at runtime), this form of explicit delegation could be not only annoying but also wrong.

The second reason is that in general you need to deal with multiple inheritance, where you do not know a priori which parent class implements the original form of the method you are overriding.

To solve these issues, Python supplies the `super()` built-in function, that climbs the class hierarchy and returns the correct class that shall be called. The syntax for calling `super()` is

In [44]:
class SecurityDoor(Door):
    colour = 'gray'
    locked = True
    
    def open(self):
        if self.locked:
            return
        super().open()

The output of `super()` is not exactly the `Door` class. It returns a `super` object which representation is `<super: <class 'SecurityDoor'>, <SecurityDoor object>>`. This object however acts like the parent class, so you can safely ignore its custom nature and use it just like you would do with the `Door` class in this case.

## Enter the Composition

Composition means that an object knows another object, and explicitly delegates some tasks to it. While inheritance is implicit, composition is explicit: in Python, however, things are far more interesting than this =).

First of all let us implement classic composition, which simply makes an object part of the other as an attribute
> 합성(구성) 방식 : 상속을 대신할 수 있다.<br/>
> 원래 상속할 클래스를 인스턴스로 집어넣는다.

In [9]:
class SecurityDoor:
    colour = 'gray'
    locked = True
    
    def __init__(self, number, status):
        self.door = Door(number, status)
        
    def open(self):
        if self.locked:
            return
        self.door.open() ## 일일이 다 바꿔야하나? / attribute error -> __getattr__ 쓰자
        
    def close(self):
        self.door.close()

The primary goal of composition is to relax the coupling between objects. This little example shows that now `SecurityDoor` is an `object` and no more a `Door`, which means that the internal structure of `Door` is not copied. For this very simple example both `Door` and `SecurityDoor` are not big classes, but in a real system objects can very complex; this means that their allocation consumes a lot of memory and if a system contains thousands or millions of objects that could be an issue.

The composed `SecurityDoor` has to redefine the `colour` attribute since the concept of delegation applies only to methods and not to attributes, doesn't it?

Well, no. Python provides a very high degree of indirection for objects manipulation and attribute access is one of the most useful. As you already discovered, accessing attributes is ruled by a special method called `__getattribute__()` that is called whenever an attribute of the object is accessed. Overriding `__getattribute__()`, however, is overkill; it is a very complex method, and, being called on every attribute access, any change makes the whole thing slower.

The method we have to leverage to delegate attribute access is `__getattr__()`, which is a special method that is called whenever the requested attribute is not found in the object. So basically it is the right place to dispatch all attribute and method access our object cannot handle. The previous example becomes

In [18]:
class X:
    pass
#     def __getattr__(self,attr):
#         return '문근영'

In [19]:
a = X()

In [20]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [21]:
a.a

AttributeError: 'X' object has no attribute 'a'

In [25]:
class X:
    def __getattr__(self,attr): # `.`로 접근할 값이 없으면 호출
        return '문근영'

In [27]:
a = X()

In [28]:
a.a

'문근영'

In [29]:
class X:
    def __getattr__(self,attr):
        if attr == 'a':
            return '문근영2'
        return '문근영'

In [30]:
a = X()
a.a

'문근영2'

In [3]:
vars(SecurityDoor)

mappingproxy({'__module__': '__main__',
              'locked': True,
              '__init__': <function __main__.SecurityDoor.__init__(self, number, status)>,
              'open': <function __main__.SecurityDoor.open(self)>,
              '__getattr__': <function __main__.SecurityDoor.__getattr__(self, attr)>,
              '__dict__': <attribute '__dict__' of 'SecurityDoor' objects>,
              '__weakref__': <attribute '__weakref__' of 'SecurityDoor' objects>,
              '__doc__': None})

In [31]:
class SecurityDoor:
    locked = True
    
    def __init__(self, number, status):
        self.door = Door(number, status)
        
    def open(self): # 바꾸고 싶다
        if self.locked:
            return
        self.door.open()
        
    def __getattr__(self, attr): #  attribute error 나오면 실행
        return getattr(self.door, attr)

In [32]:
a = 1.0
getattr(a, 'hex')()

'0x1.0000000000000p+0'

`__getattr__` : 값이 없을 때

`getattr(instance, attr)` : 해당 인스턴스의 attr 가져온다. 

In [33]:
try:
    a = 1/0
except:
    a = 2/0

ZeroDivisionError: division by zero

In [ ]:
class SecurityDoor:
    locked = True
    
    def __init__(self, number, status):
        self.door = Door(number, status)
        
    def open(self): # 바꾸고 싶다
        if self.locked:
            return
        self.door.open()
        
    def __getattribute__(self, attr): # __getattr__전에 실행됨
        print('문')
        
    def __getattr__(self, attr): #  attribute error 나오면 실행
        return getattr(self.door, attr)

Using `__getattr__()` blends the separation line between inheritance and composition since after all the former is a form of automatic delegation of every member access.

In [35]:
class ComposedDoor:
    def __init__(self, number, status):
        self.door = Door(number, status)
        
    def __getattr__(self, attr):
        return getattr(self.door, attr)

> - 상속 안쓰고 상속할 애를 인스턴스로 내 클래스 정의 안에 집어넣는다.<br/>
> - 내가 수정할 부분은 재정의한다.
> - 수정하지 않을 부분은 __getattr__() 문 쓴다

In [38]:
a = ComposedDoor(1, '1')

In [39]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'door']

As this last example shows, delegating every member access through `__getattr__()` is very simple. Pay attention to `getattr()` which is different from `__getattr__()`. The former is a built-in that is equivalent to the dotted syntax, i.e. `getattr(obj, 'someattr')` is the same as `obj.someattr`, but you have to use it since the name of the attribute is contained in a string.

Composition provides a superior way to manage delegation since it can selectively delegate the access, even mask some attributes or methods, while inheritance cannot. In Python you also avoid the memory problems that might arise when you put many objects inside another; Python handles everything through its reference, i.e. through a pointer to the memory position of the thing, so the size of an attribute is constant and very limited.

## Movie Trivia

Section titles come from the following movies: _The Cannonball Run (1981)_, _Apocalypse Now (1979)_, _Enter the Dragon (1973)_.

## Sources

You will find a lot of documentation in [this Reddit post](http://www.reddit.com/r/Python/comments/226ahl/some_links_about_python_oop/). Most of the information contained in this series come from those sources.

## Feedback

Feel free to use [the blog Google+ page](https://plus.google.com/u/0/b/110554719587236016835/110554719587236016835/posts) to comment the post. The [GitHub issues](https://github.com/lgiordani/lgiordani.github.com/issues) page is the best place to submit corrections.
